In [3]:
import numpy as np
import pandas as pd
import string
import email
import os ##use this to traverse folders
from email.parser import Parser

#extract labels from TREC2007/full/ file lists
with open('trec07p/full/index', 'r') as ifile:
    raw_labels = ifile.readlines()

#using file list to give labels and e-mail numbers

categories = [i.split( )[0] for i in raw_labels]
email_nums = [i.split(".")[-1][:-1] for i in raw_labels]
labels = pd.Series(categories, index=email_nums)


IOError: [Errno 2] No such file or directory: 'trec07p/full/index'

In [ ]:
#reference from other place

# parser = Parser()
# corpus = []
# for email_num in email_nums:
#     filepath = 'trec07p/data/inmail.%s' % email_num

#     with open(filepath, 'r',) as ifile:
#         email = parser.parse(ifile)

#     ## TODO also keep the subject header

#     ## code and details from:
#     ## https://stackoverflow.com/questions/17874360/python-how-to-parse-the-body-from-a-raw-email-given-that-raw-email-does-not/32840516#32840516
#     if email.is_multipart():
#         for part in email.walk():
#             content_type = part.get_content_type()
#             content_dispo = str(part.get('Content-Disposition'))

#             # skip any text/plain (txt) attachments
#             if content_type == 'text/plain' and 'attachment' not in content_dispo:
#                 body = part.get_payload(decode=False)
#                 break ## only keep the first email
#     else:
#         body = email.get_payload(decode=False)

#     ## TODO strip html stuff ? or valuable features ?
#     try:
#         corpus.append((email_num, body))
#     except NameError:
#         pass

# email_nums, corpus = zip(*corpus)

In [ ]:
# extract the body of email
def walkMsg(msg):
    """
    Input: string of email
    Output: body of email
    """
    for part in msg.walk():
        if part.get_content_type() == "multipart/alternative":
            continue
        yield part.get_payload(decode=1)

In [ ]:
def info_of_email_2():
    """
    Input: path, list of files name, classification(spam or ham)
    Ourput: a list with info of email
    """
    
    parser = Parser()
    info = []
    for nums in range(len(categories)):
        filepath = 'trec07p/data/inmail.%s' % email_nums[nums]
        
        with open(filepath, 'r',) as ifile:
            email = parser.parse(ifile)
            info.append([email.get("From"), email.get('To'), email.get('cc'), email.get('bcc'), 
                         str(email.get("Content-Type")).split(";")[0],
                         email.get('subject'), " ".join([i for i in walkMsg(email) if i != None]),categories[nums]])
    return info

In [4]:
df = pd.DataFrame(info_of_email_2(),
                  columns=['From','To','cc','bcc','Content_type','Subjects','Content','Classification'])

NameError: name 'info_of_email_2' is not defined

In [ ]:
df.head()

In [ ]:
##encode
def robust_decode(bs):
    '''Takes a byte string as param and convert it into a unicode one.
First tries UTF8, and fallback to Latin1 if it fails'''
    cr = None
    try:
        cr = bs.decode('utf8')
    except UnicodeDecodeError:
        cr = bs.decode('ISO-8859-1')
    return cr
df['Content'] =[robust_decode(i) for i in df['Content']]
##remove newlines break/return
df['Content']=[i.replace('\n','').replace('\r','').replace('\t','') for i in df['Content']]

In [ ]:
##remove html tag
#import lxml.html

#df[['html' in i for i in df['Content_type']]]['Content'] = [lxml.html.fromstring(k).text_content() for k in df[['html' in i for i in df['Content_type']]]['Content']]